# Исследование надежности заемщиков


Во второй части проекта вы выполните шаги 3 и 4. Их вручную проверит ревьюер.
Чтобы вам не пришлось писать код заново для шагов 1 и 2, мы добавили авторские решения в ячейки с кодом. 



## Откройте таблицу и изучите общую информацию о данных

**Задание 1. Импортируйте библиотеку pandas. Считайте данные из csv-файла в датафрейм и сохраните в переменную `data`. Путь к файлу:**

`/datasets/data.csv`

In [ ]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Задание 2. Выведите первые 20 строчек датафрейма `data` на экран.**

In [ ]:
data.head(20)

**Задание 3. Выведите основную информацию о датафрейме с помощью метода `info()`.**

In [ ]:
data.info()

## Предобработка данных

### Удаление пропусков

**Задание 4. Выведите количество пропущенных значений для каждого столбца. Используйте комбинацию двух методов.**

In [ ]:
data.isna().sum()

**Задание 5. В двух столбцах есть пропущенные значения. Один из них — `days_employed`. Пропуски в этом столбце вы обработаете на следующем этапе. Другой столбец с пропущенными значениями — `total_income` — хранит данные о доходах. На сумму дохода сильнее всего влияет тип занятости, поэтому заполнить пропуски в этом столбце нужно медианным значением по каждому типу из столбца `income_type`. Например, у человека с типом занятости `сотрудник` пропуск в столбце `total_income` должен быть заполнен медианным доходом среди всех записей с тем же типом.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Задание 6. В данных могут встречаться артефакты (аномалии) — значения, которые не отражают действительность и появились по какой-то ошибке. таким артефактом будет отрицательное количество дней трудового стажа в столбце `days_employed`. Для реальных данных это нормально. Обработайте значения в этом столбце: замените все отрицательные значения положительными с помощью метода `abs()`.**

In [ ]:
data['days_employed'] = data['days_employed'].abs()

**Задание 7. Для каждого типа занятости выведите медианное значение трудового стажа `days_employed` в днях.**

In [ ]:
data.groupby('income_type')['days_employed'].agg('median')

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Задание 8. Выведите перечень уникальных значений столбца `children`.**

In [ ]:
data['children'].unique()

**Задание 9. В столбце `children` есть два аномальных значения. Удалите строки, в которых встречаются такие аномальные значения из датафрейма `data`.**

In [ ]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Задание 10. Ещё раз выведите перечень уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [ ]:
data['children'].unique()

### Удаление пропусков (продолжение)

**Задание 11. Заполните пропуски в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [ ]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Задание 12. Убедитесь, что все пропуски заполнены. Проверьте себя и ещё раз выведите количество пропущенных значений для каждого столбца с помощью двух методов.**

In [ ]:
data.isna().sum()

### Изменение типов данных

**Задание 13. Замените вещественный тип данных в столбце `total_income` на целочисленный с помощью метода `astype()`.**

In [ ]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Задание 14. Обработайте неявные дубликаты в столбце `education`. В этом столбце есть одни и те же значения, но записанные по-разному: с использованием заглавных и строчных букв. Приведите их к нижнему регистру. Проверьте остальные столбцы.**

In [ ]:
data['education'] = data['education'].str.lower()

**Задание 15. Выведите на экран количество строк-дубликатов в данных. Если такие строки присутствуют, удалите их.**

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

### Категоризация данных

**Задание 16. На основании диапазонов, указанных ниже, создайте в датафрейме `data` столбец `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.


**Например, кредитополучателю с доходом 25000 нужно назначить категорию `'E'`, а клиенту, получающему 235000, — `'B'`. Используйте собственную функцию с именем `categorize_income()` и метод `apply()`.**

In [ ]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [ ]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Задание 17. Выведите на экран перечень уникальных целей взятия кредита из столбца `purpose`.**

In [ ]:
data['purpose'].unique()

**Задание 18. Создайте функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

**Например, если в столбце `purpose` находится подстрока `'на покупку автомобиля'`, то в столбце `purpose_category` должна появиться строка `'операции с автомобилем'`.**

**Используйте собственную функцию с именем `categorize_purpose()` и метод `apply()`. Изучите данные в столбце `purpose` и определите, какие подстроки помогут вам правильно определить категорию.**

In [ ]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [ ]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Шаг 3. Исследуйте данные и ответьте на вопросы

#### 3.1 Есть ли зависимость между количеством детей и возвратом кредита в срок?

Посмотрим первые 20 строк получившейся после предобработки таблицы:

In [ ]:
data.head(20)

In [ ]:
data.head()

Посмотрим общую информацию по таблице:

In [ ]:
data.info()

Посмотрим как распределяются данные в таблице в зависимости от наличия детей и их колличества: 

In [ ]:
data.groupby('children')['debt'].count()

Видим что больше всего кредитов у заемщиков не имеющих детей. Не спешим с выводом так как заемщиков без детей может быть просто больше в таблице и правильней будет проводить подсчеты в двух группах заемщикв с детьми и без детей.

Посчитаем количество заемщиков в разных группах: 

In [ ]:
# Подсчеты для заемщиков с детьми
childok = data[data['children'] > 0]
childok_len_total = len(childok['children'])
childok_len =childok.loc[((childok['children'] > 0) &(childok['debt' ] > 0))]
childok_len_1 = len(childok_len['children'])
# Подсчеты для заемщиков без детей
childfree = data[data['children'] == 0]
childfree_len = len(childfree['children'])
childfree_len_debt = childfree.loc[((childfree['children'] == 0) &(childfree['debt' ] > 0))]
childfree_len_debt_1 = len(childfree_len_debt['children']) 

#Подсчеты для заемщиков с 1 и более детей
childok_1 = childok.loc[((childok['children'] == 1) &(childok['debt' ] > 0))]
childok_1_len = len(childok_1['children'])
childok_2 = childok.loc[((childok['children'] > 1) &(childok['debt' ] > 0))]
childok_2_len = len(childok_2['children'])

print('Общее количество заемщиков с детьми:', childok_len_total)
print('Общее количество заемщиков без детей:',childfree_len)
print('обще количество заемщиков без детей и с просрочками по кредиту:',childfree_len_debt_1,)
print('Общее количество заемщиков с детьми и просрочками по кредиту:', childok_len_1)
print('Общее колличество заемщиков с 1 ребенком и просрочками по кредиту:',childok_1_len)
print('Общее количество заемщиков с 2 и более детьми и просрочками по кредиту:',childok_2_len)

Посчитаем процент кредитов с просрочками среди заемщиков с детьми:

In [ ]:
child_ok = childok_len_1/childok_len_total*100
child_ok

Посчитаем процент кредитов с просрочками среди заемщиков без детей:

In [ ]:
childfree_debt = childfree_len_debt_1/childfree_len*100
childfree_debt

Посчитаем влияние колличества детей на просрочки по кредитам разбив должников с детьми на 2 группы имеющие 1 ребенка и 2х и более.


Расчет для группу заемщиков с 1 ребенком:

In [ ]:
childok_1_debt = childok_1_len/childok_len_total*100
childok_1_debt

Расчет для группы с 2 и более детьми:

In [ ]:
childok_2_debt = childok_2_len/childok_len_total*100
childok_2_debt

 **Вывод:** Гипотеза подтвердилась. Так заемщики имеющие детей чаще допускают просрочки 9.24% против 7.54% среди замещиков не имеющих детей. Проверка гипотезы в отношении влияния колличества детей так же частично подтвердилась. Так заемщики имеющие 1 ребенка допускают просрочку в 6.13% случаев а имеющие 2 и более детей 3.11%. Результаты показывают снижение процента должников с количесвтом детей. Возможно дело в количестве заемщиков 1 и 2 группы так 1 группа имеющая 1 ребенка в 2 раза больше 2 группы заемщиков с 2 и более детьми 444 и 225 соответственно. Но в целом расчеты показывают что должники с детьми менее дисциплинированные заемщики.

#### 3.2 Есть ли зависимость между семейным положением и возвратом кредита в срок?

Посмотрим как распределяются данные в таблице среди должников по статусам семейного положения.

In [ ]:
data.groupby(by = 'family_status' )['debt'].count()

In [ ]:
data.groupby(by = 'family_status_id' )['debt'].count()

Предварительные данные говорят, что больше всего кредитов среди женатых заемщиков. А заемщики в таблице разбиты на 5 категорий которые по смыслу можно обьединить в 2

Посчитаем общее колличество должников со статусом в браке это 2 группы 'женат / замужем' и 'гражданский брак'.

In [ ]:
family_status_on = data[data['family_status_id'] <=1]
family_status_on_len = len(family_status_on['family_status_id'])
print (family_status_on_len)
family_status_on_debt = family_status_on.loc[((family_status_on['family_status_id'] <=1) &(family_status_on['debt' ] > 0))]
family_status_on_debt_len = len(family_status_on_debt['family_status_id'])
print(family_status_on_debt_len)


Получили 16395 заемщиков со статусом в браке (официальный или гражданский) и 1312 заемщиков состоящие в браке и имеющие задолженность осталось посчитать отношение в процентах:

In [ ]:
family_status_on_debt_len/family_status_on_len*100

Посчитаем общее колличество должников со статусом не в браке:

In [ ]:
family_status_off = data[data['family_status_id'] >=2]
family_status_off_len = len(family_status_off['family_status_id'])
print(family_status_off_len)
family_status_off_debt = family_status_off.loc[((family_status_off['family_status_id'] >=2) &(family_status_off['debt' ] > 0))]
family_status_off_debt_len = len(family_status_off_debt['family_status_id'])
print(family_status_off_debt_len)


Получили 4936 заемщиков со статусом не в браке (в разводе, вдовец / вдова, Не женат / не замужем) и 420 заемщиков не в браке и имеющие задолженность. Осталось посчитать отношение в процентах:
    
    
    

In [ ]:
family_status_off_debt_len/family_status_off_len*100

**Вывод:** Гипотеза подтвердилась зависимость от семейного положения небольшая но есть. Так заемщики не в браке имеют просрочки по кредитам в 8.5% в отличии от должников состоящих в браке 8.0% соответственно.

#### 3.3 Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

Посмотрим как распределяются данные по категориям уровня дохода:

In [ ]:
data.groupby('total_income_category')['debt'].count()

Больше всего займов берут люди с уровнем дохода соответствующий категории 'С'. Посчитаем процент просрочек в каждой группе заемщиков и их количество:
    

In [ ]:
#Подсчет для категории "А"
total_income_debt_a = data.loc[((data['total_income_category'] == 'A') &(data['debt' ] > 0))]
total_income_debt_a_len = len(total_income_debt_a['total_income_category'])
print('В категории А просрочек:', total_income_debt_a_len, 'процент не просрочек', total_income_debt_a_len/25*100)

#Подсчет для категории "B"
total_income_debt_b = data.loc[((data['total_income_category'] == 'B') &(data['debt' ] > 0))]
total_income_debt_b_len = len(total_income_debt_b['total_income_category'])
print('В категории B просрочек:', total_income_debt_b_len, 'процент просрочек', total_income_debt_b_len/5014*100)

#Подсчет для категории "C"
total_income_debt_c = data.loc[((data['total_income_category'] == 'C') &(data['debt' ] > 0))]
total_income_debt_c_len = len(total_income_debt_c['total_income_category'])
print('В категории C просрочек:', total_income_debt_c_len, 'процент просрочек', total_income_debt_c_len/15921*100)

#Подсчет для категории "D"
total_income_debt_d = data.loc[((data['total_income_category'] == 'D') &(data['debt' ] > 0))]
total_income_debt_d_len = len(total_income_debt_d['total_income_category'])
print('В категории D просрочек:', total_income_debt_d_len,'процент просрочек', total_income_debt_d_len/349*100)

#Подсчет для категории "E"
total_income_debt_e = data.loc[((data['total_income_category'] == 'E') &(data['debt' ] > 0))]
total_income_debt_e_len = len(total_income_debt_e['total_income_category'])
print('В категории E просрочек:', total_income_debt_e_len,'процент просрочек', total_income_debt_e_len/22*100)

Произведем подсчет процентного отношения кредитов с просрочками к общему чеслу кредитов в указанных группах:

Для наглядности поместим полученные данные в таблицу:

In [ ]:
df = pd.DataFrame({'total_income_category':["A","B","C","D","E"], 'total_income_category_count':[25,5014,15921,349,22], 'total_income_category_debt_count': [2,354,1353,21,2], 'total_income_category_debt_procent':[8.0,7.06,8.49,6.01,9.09]})
df
                   
                   

**Вывод:** Гипотеза подтвердилась. в зависимости от категории дохода заемщика процент займов с просрочками разнится. Так наименее дисциплинированные заемщики в категории 'E' а самые дисципленированные в категории 'D'

#### 3.4 Как разные цели кредита влияют на его возврат в срок?

Посмотри как распределяются данные в таблице по целям кредита ранее мы их распределили на 4 группы:

In [ ]:
data.groupby('purpose_category')['debt'].count()

Видим что больше всего займов берут на операции с недвижимостью.Посчитаем процент просрочек в каждой группе заемщиков и их количество:

In [ ]:
#Подсчет для категории "операции с автомобилем"
purpose_category_auto_debt = data.loc[((data['purpose_category'] == 'операции с автомобилем') &(data['debt' ] > 0))]
purpose_category_auto_debt_len = len(purpose_category_auto_debt['purpose_category'])
print('В категории операции с автомобилем просрочек:', purpose_category_auto_debt_len, 'процент просрочек', purpose_category_auto_debt_len/4279*100)

#Подсчет для категории "операции с недвижимостью"
purpose_category_property_debt = data.loc[((data['purpose_category'] == 'операции с недвижимостью') &(data['debt' ] > 0))]
purpose_category_property_debt_len = len(purpose_category_property_debt['purpose_category'])
print('В категории операции с недвижимостью просрочек:', purpose_category_property_debt_len, 'процент просрочек', purpose_category_property_debt_len/10751*100)

#Подсчет для категории "получение образования"
purpose_category_education_debt = data.loc[((data['purpose_category'] == 'получение образования') &(data['debt' ] > 0))]
purpose_category_education_debt_len = len(purpose_category_education_debt['purpose_category'])
print('В категории получение образования просрочек:', purpose_category_education_debt_len, 'процент просрочек', purpose_category_education_debt_len/3988*100)

#Подсчет для категории "проведение свадьбы"
purpose_category_wedding_debt = data.loc[((data['purpose_category'] == 'проведение свадьбы') &(data['debt' ] > 0))]
purpose_category_wedding_debt_len = len(purpose_category_wedding_debt['purpose_category'])
print('В категории проведение свадьбы просрочек:', purpose_category_wedding_debt_len, 'процент просрочек', purpose_category_wedding_debt_len/2313*100)



Для наглядности поместим данные в таблицу:

In [ ]:
df_target = pd.DataFrame({'purpose_category':["операции с автомобилем","операции с недвижимостью","получение образования","проведение свадьбы"], 'purpose_category_count':[4279,10751,3988,2313], 'purpose_category_debt_count': [400,780,369,183], 'purpose_category_debt_procent':[9.34,7.25,9.25,7.91]})
df_target

**Вывод:** Гипотеза подтвердилась цель кредита влияет на процент займов с просрочами. Так заемщики в категории операции с недвижимостью самые дисциплинированные, а в категории операции с автомобилем самые не дисциплинированные.

#### 3.5 Приведите возможные причины появления пропусков в исходных данных.

*Ответ:* Пропуски в исходных данных могут быть случайными, не сучайными, вызванными техническими ошибками, ошибками кода, или ошибками ввода.

#### 3.6 Объясните, почему заполнить пропуски медианным значением — лучшее решение для количественных переменных.

*Ответ:* Медианное значение менее подверженно резким выбросам в данных, а соответсвенно позволит произвести посдсчеты более точно.

### Шаг 4: общий вывод.

Перед исследованием стояла задача заказчика — кредитный отдел банка. Нужно было понять, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Для этого мы проверили следующие гипотезы: 

-Есть ли зависимость между количеством детей и возвратом кредита в срок:
Проверка данной гипотезы показала что зависимость есть и колличество детей влияет на просрочки по кредитам. Так заемщики имеющие детей чаще допускают просрочки 9.24% против 7.54% среди замещиков не имеющих детей. Проверка гипотезы в отношении влияния колличества детей так же частично подтвердилась. Так заемщики имеющие 1 ребенка допускают просрочку в 6.13% случаев, а имеющие 2 и более детей 3.11%.

-Есть ли зависимость между семейным положением и возвратом кредита в срок:
Гипотеза подтвердилась зависимость от семейного положения есть. Так заемщики не в браке имеют просрочки по кредитам в 8.5% в отличии от должников состоящих в браке 8.0% соответственно. Соответствено данные о семейном положении так же нужно учитывать при проведении кредитного скоринга.

-Есть ли зависимость между уровнем дохода и возвратом кредита в срок:
Гипотеза подтвердилась. в зависимости от категории дохода заемщика процент займов с просрочками разнится. Так наименее дисциплинированные заемщики в категории 'E' а самые дисципленированные в категории 'D' Соответственно без этих данных кредитный скорин потеряет в точности.

-Как разные цели кредита влияют на его возврат в срок:

Гипотеза подтвердилась цель кредита влияет на процент займов с просрочами. Так заемщики в категории операции с недвижимостью самые дисциплинированные, а в категории операции с автомобилем самые не дисциплинированные.

Для проведения правильных расчетов данные были очищенны от дубликатов, пропусков, разбиты на смысловые категории (на пример по уникальным целям взятия кредита), убраны выбросы что позволило сделать выводы точными и достоверными. 

Подводя общий итог можно утверждать, что все исследованные гипотезы подтвердились и заказчику следует использовать данные показатели при осуществлении кредитного скоринга.
